# Context

We don't need more tools that show use more problems in our software. We need ways to determine the right problem to solve. But often we just fix the symptoms on the surface rather then the underlying problems. I find that approach non-professional and want to do my part to improve this situation by delivering root cause analysis of symptoms to get to the real problem.

In this notebook, I'll show you one of my approaches of mining performance problems based on application runtime analysis. One could use such an analysis to make the point that there are severe design errors that have negative influence on the application's overall performance.

The key idea is to use graph analysis to analyze performance profiling call stacks. With this approach I not only want to show what I call "the fact" that is causing a performance issue (because the fact shows that something takes long to execute or is executed to many times), but also the reasons behing the fact. I try to do this by extracting various additional information like the application's entry points, the trigger within the application causing the fact and the immediate reason that calls the fact. This may sound a little theoretical, but I'll show the approach step by step.

# Profiling with JProfiler
As a prerequisit for this analysis, we need performance profiling data gathered by a profiler or an Application Performance Management tool. In my case, I run a performance test with JMeter for a [fork of the Spring sample project PetClinic](https://buschmais.github.io/spring-petclinic/) and use [JProfiler](http://www.ej-technologies.com/products/jprofiler/overview.html) to record some performance measures while the test was running.  

<p><tt>&lt;advertisment&gt;</tt><br />
JProfiler is a great commercial tool and costs around 400 €. It really worth the money because it gives you deep insights how your application performs under the hood. At this point, I want to thank ej-technologies for giving me with a [free open-source license](https://www.ej-technologies.com/buy/jprofiler/openSource) for JProfiler that enables this blog post in exechange of mentioning their product:

<a href="http://www.ej-technologies.com/products/jprofiler/overview.html">
![](https://www.ej-technologies.com/images/product_banners/jprofiler_large.png)</a>
<tt>&lt;/advertisment&gt;</tt>
</p>

What we need for this analysis is the recorded runtime stack of the application calles a call tree. A call tree shows you a tree of the called methods. Some tools do this on a pre-requeset basis (like XRebel, StageMonitor, InspectIT or DynaTrace) or for the time of the recording (like JProfiler or YourKit). So below, you can see the call tree for the called methods with their measured CPU wall clock time (aka the real time that is spent in that method) for a complete test run:

![](resources/jprofiler_cpu_call_tree.png)

It shows you the call hierarchy from entry point into an application to the last called method (the "leafs") as well as the time in that method ("self-time") and the number of invocations:

![](resources/jprofiler_cpu_call_tree_leaf.png)

With such a view, you see which parts of your application call which classes and methods by drilling down the hierarchy. But there is more: You can also "turn around" the call tree and listen all the so called "HotSpots". Technically, e. g. for CPU HotSpots, JProfiler sums up all the measurements for the method call leafs that take longer than 0.1% of all method calls:

![](resources/jprofiler_cpu_hot_spots.png)

These views are also available for other measures like web service calls, file accesses or DB calls, that is shown below:

![](resources/jprofiler_jdbc_hot_spots.png)

In the JProfiler GUI (or the command line tool jpexport), you can export such a call tree into a XML file. And this is were our journey begins.

# Reading XML into Neo4j

In [ ]:
If you export the HotSpots of some measures with 

In [3]:
with open (r'data/spring-petclinic/JDBC_Probe_Hot_Spots.xml') as log:
    [print(line[:120] + "...") for line in log.readlines()[:10]]

<?xml version="1.0" encoding="UTF-8"?>
...
<tree type="Probe Hot Spots" probe="JDBC" threadSelection="All thread groups" threadStatus="All states" aggregationLevel...
  <hotspot leaf="false" value="SELECT COUNT(*) FROM INFORMATION_SCHEMA.SYSTEM_USERS" time="598631" count="26">
...
    <node leaf="false" class="org.hsqldb.jdbc.JDBCStatement" methodName="execute" methodSignature="(Ljava/lang/String;)Z...
      <node leaf="false" class="java.sql.Statement" methodName="execute" methodSignature="(Ljava/lang/String;)Z" time="5...
        <node leaf="false" class="org.apache.tomcat.jdbc.pool.PooledConnection" methodName="validate" methodSignature="(...
          <node leaf="false" class="org.apache.tomcat.jdbc.pool.PooledConnection" methodName="validate" methodSignature=...
            <node leaf="false" class="org.apache.tomcat.jdbc.pool.ConnectionPool" methodName="borrowConnection" methodSi...
              <node leaf="false" class="org.apache.tomcat.jdbc.pool.ConnectionPool" methodName="bo

Die folgende Datei wurde über jQAssistant

<pre>
jqassistant scan -f xml:document::JDBC_Probe_Hot_Spots.xml
</pre>

in Neo4J eingelesen.

 Der Neo4J-Server wurde mit 

<pre>
jqassistant server
</pre>

mit den Standardeinstellungen gestartet.

Wichtig ist, dass der Server unter der Oracle JVM läuft, was z. B. mit
<pre>
set PATH=C:\Development\apps\Java\jdk1.8.0_25\bin;%PATH%
</pre>
gesetzt werden kann.

# Erstelle Konten für eine bessere Analyse

In [4]:
from py2neo import Graph
graph = Graph()
from py2neo.packages.httpstream import http
http.socket_timeout = 240

## 0. Vorbereitung

### Lösche alle evtl. vorhandenen Konten und Beziehungen

In [5]:
query = """
MATCH (c)-[r:CALLS]->()
DELETE r, c
RETURN COUNT(r), count(c)
"""
graph.run(query).data()

[{'COUNT(r)': 0, 'count(c)': 0}]

### Setze Index

In [6]:
query = """
CREATE INDEX ON :Element(name)
"""
#graph.run(query).data()

## 1. Call-Nodes aus XML-Elements erzeugen

In [7]:
query = """
MATCH (n:Element {name: "node"}),
    (n)-[:HAS_ATTRIBUTE]->(classAttribut:Attribute {name : "class"}),
    (n)-[:HAS_ATTRIBUTE]->(methodAttribut:Attribute {name : "methodName"}),
    (n)-[:HAS_ATTRIBUTE]->(countAttribut:Attribute {name : "count"}),
    (n)-[:HAS_ATTRIBUTE]->(timeAttribut:Attribute {name : "time"})
CREATE 
   (x:Node:Call {
        class: classAttribut.value, 
        method: methodAttribut.value,
        count: toFloat(countAttribut.value), 
        time: toFloat(timeAttribut.value)
        })-[r:CREATED_FROM]->(n)
RETURN COUNT(x), COUNT(r)
"""
graph.run(query).data()

[{'COUNT(r)': 8349, 'COUNT(x)': 8349}]

In [8]:
query = """
MATCH (n:Element { name: "hotspot"}),
    (n)-[:HAS_ATTRIBUTE]->(valueAttribut:Attribute {name : "value"}),
    (n)-[:HAS_ATTRIBUTE]->(countAttribut:Attribute {name : "count"}),
    (n)-[:HAS_ATTRIBUTE]->(timeAttribut:Attribute {name : "time"})
    
WHERE n.name = "hotspot"
CREATE 
   (x:Node:HotSpot {
        value: valueAttribut.value,
        count: toFloat(countAttribut.value), 
        time: toFloat(timeAttribut.value)
        })-[r:CREATED_FROM]->(n)
RETURN COUNT(x), COUNT(r)
"""
graph.run(query).data()

[{'COUNT(r)': 87, 'COUNT(x)': 87}]

## 2. Beziehung zu allen Calls untereinander herstellen

In [9]:
query="""
MATCH (outerCall:Call)-[:CREATED_FROM]->(outerNode)-[:HAS_ELEMENT]->(innerNode)<-[:CREATED_FROM]-(innerCall:Call)
CREATE (outerCall)<-[r:CALLS]-(innerCall)
RETURN COUNT(r)
"""
graph.run(query).data()

[{'COUNT(r)': 8262}]

## 4. Hotspots mit Calls verbinden

In [10]:
query="""
MATCH (outerCall:HotSpot)-[:CREATED_FROM]->(outerNode)-[:HAS_ELEMENT]->(innerNode)<-[:CREATED_FROM]-(innerCall:Call)
CREATE (outerCall)<-[r:CALLS]-(innerCall)
RETURN COUNT(r)
"""
graph.run(query).data()

[{'COUNT(r)': 87}]

In [11]:
query = """
MATCH ()-[r:CREATED_FROM]->(:Element)
DELETE r
RETURN COUNT(r)
"""
graph.run(query).data()

[{'COUNT(r)': 8436}]

# Analysen

In [12]:
import pandas as pd

## TOP-Auslöser der HotSpots

In [32]:
query="""
MATCH (request:Call)-[:CALLS*]->(entry:Call)-[:CALLS*1..99]->(trigger:Call)-[:CALLS*]->(hotspot:HotSpot)
WHERE 
    request.class = "_jprofiler_annotation_class" AND
    entry.class STARTS WITH "org.springframework.samples.petclinic." AND
    trigger.class STARTS WITH "org.springframework.samples.petclinic."

RETURN
    request.method as request, 
    request.count as count, 
    SPLIT(entry.class, ".")[-1] as entry_class,
    SPLIT(entry.method, ".")[-1] as entry_method,    
    SPLIT(trigger.class, ".")[-1] as trigger_class,    
    SPLIT(trigger.method, ".")[-1] as trigger_method,    
    hotspot.value as sql, 
    hotspot.count as count_sum
ORDER BY count DESC
"""
auswertung = pd.DataFrame(graph.run(query).data())

KeyError: 'a_id'

In [14]:
auswertung.iloc[0]['class']

KeyError: 'class'

In [ ]:
df['bk'] = df['b'].apply(lambda x : x.split("#")[0])
df.head()

In [ ]:
grouped= df.groupby('bk').sum().sort_values(by="Zeitdauer", ascending=False)
grouped.head()

In [ ]:
top = grouped.head(10)
top 

In [ ]:
andere_hotspots = grouped[~grouped['Zeitdauer'].isin(top.index)]
andere_hotspots.head()

In [ ]:
name_andere = "" + str(len(andere_hotspots)) + " weitere Klassen"
andere_hotspots_summiert = pd.DataFrame(andere_hotspots.sum(), columns=[name_andere]).T
andere_hotspots_summiert.head()

In [ ]:
ergebnis_top10_hotspots = top.append(andere_hotspots_summiert)
ergebnis_top10_hotspots['Verweildauer in Minuten'] = top['Zeitdauer'].apply(
    lambda p: '{:.2f}'.format(p/1000/1000/60))
ergebnis_top10_hotspots.index.name="Klassenname"
ergebnis_top10_hotspots[['Verweildauer in Minuten']]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np

def plotteDaten(series, titel):

    plt.style.use('fivethirtyeight')
    plt.figure(facecolor='white')

    ax = series.plot(
        kind='pie', 
        colors=cm.Spectral(np.linspace(0, 1, len(series))), 
        figsize=(10,10), 
        legend=None, 
        autopct='%1.0f%%',
        pctdistance=1.1, 
        labeldistance=1.2,
        fontsize=10,
        title=titel)
    ax.set_ylabel("")
    fig = ax.get_figure()
    plt.savefig("output/" + titel + ".svg", facecolor=fig.get_facecolor(), bbox_inches="tight")

In [ ]:
plotteDaten(top['Zeitdauer'], u"Cloner-Verweildauer")